# Table of content:
## 01. Importing the Data and Libraries
## 02. Security Implications: Addressing Personally Identifiable Information (PII)
## 03. Creating a regional segmentation of the data - “Region” column based on the “State” column from the customer's data set.
## 04. Creating an exclusion flag for low-activity customers (customers with less than 5 orders) and excluding them from the data.
## 05. Creating a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents.

# 01. Importing the Data and Libraries 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#Path 
path = r'C:\Users\ideal\OneDrive\Documents\Task 4.6.7.8.9.10'

In [4]:
# importing dataframe merged_final_df.pkl (Final merged cleaned sets)
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merged_final_df.pkl'))

In [5]:
df.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income,...,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,_merge
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Slowest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,both


In [6]:
# Dropping the "_merge" column
df = df.drop(columns = ['_merge'])

In [3]:
# Removing the limit to the amount of columns displayed by default: 
pd.set_option('display.max_columns', None)

# 02. Security Implications: Addressing  Personally Identifiable Information (PII)

### When it comes to PII, particularly sensitive are names, email addresses, physical addresses, and phone numbers. In our case, we have displayed the first and last names. I will get rid of these two columns.

In [10]:
# Dropping "first_name" and "last_name" column.
df = df.drop(['first_name', 'last_name'], axis=1)

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,2524893,3,3,11,30.0,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Slowest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,80148,Female,New York,55,4/1/2020,1,married,57095,2859858,3,6,11,12.0,38750,11,0,"Protein Bar, Chocolate Pretzel",3,19,4.7,Low-range product,Regularly busy,Regularly busy,Most orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer
32404855,80148,Female,New York,55,4/1/2020,1,married,57095,2859858,3,6,11,12.0,39972,5,0,Chocolate Fudge High Protein Bar,3,19,4.0,Low-range product,Regularly busy,Regularly busy,Most orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer
32404856,80148,Female,New York,55,4/1/2020,1,married,57095,3209855,4,6,19,7.0,3198,2,0,Pumpkin Spice Latte Iced Coffee Pure Cold Brew...,26,7,4.7,Low-range product,Regularly busy,Regularly busy,Average orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer
32404857,80148,Female,New York,55,4/1/2020,1,married,57095,2859858,3,6,11,12.0,39069,2,0,Chocolate Coconut Almond Protein Bar,3,19,1.8,Low-range product,Regularly busy,Regularly busy,Most orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer


# 03. Creating a regional segmentation of the data -  “Region” column based on the “State” column from the customer's data set.

### First step: creating the regions

In [13]:
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']

In [14]:
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']

In [15]:
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']

In [16]:
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

### Creating the new column with the regions created:

In [17]:
df.loc[df['state'].isin(Northeast), 'region'] = 'Northeast'

In [18]:
df.loc[df['state'].isin(Midwest), 'region'] = 'Midwest'

In [19]:
df.loc[df['state'].isin(South), 'region'] = 'South'

In [20]:
df.loc[df['state'].isin(West), 'region'] = 'West'

In [21]:
# Checking the result - New "region" column successfully created 
df['region'].value_counts(dropna = False)

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [22]:
df.head()

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,2524893,3,3,11,30.0,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Slowest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest


### Determining whether there’s a difference in spending habits between the different U.S. regions. Creating a crosstab -  crossing the newly created variable 'regions' with the previously created 'spending_flag' variable. 

In [24]:
crosstab = pd.crosstab(df['spending_flag'], df['region'], dropna = False)

In [25]:
crosstab.to_clipboard()

In [26]:
crosstab

region,Midwest,Northeast,South,West
spending_flag,,,,
High spender,29265,18642,40564,31223
Low spender,7568060,5704094,10751321,8261690


### The spending habits in different regions appear to be approximately the same, less than one percent appear to be high spenders. Looking at the total number of spending per region, the South makes up most of the spenders with around 33% followed by the  West with around 26%, the Midwest with 23 %, and 18% Northeast.

# 04. Creating an exclusion flag for low-activity customers (customers with less than 5 orders) and excluding them from the data. 


In [31]:
df.loc[df['max_order'] < 5, 'activity_flag'] = 'low activity'

In [32]:
df.loc[df['max_order'] >= 5, 'activity_flag'] = 'active customer'

In [33]:
df['activity_flag'].value_counts(dropna=False)

activity_flag
active customer    30964564
low activity        1440295
Name: count, dtype: int64

### Creating a subset of low activity customers

In [34]:
df_low_activity = df[df['activity_flag'] =='low activity']

In [39]:
df_active_customers = df[df['activity_flag'] =='active customer']

In [37]:
# The subset numbers add up, 1440295 low activity customers 
df_low_activity

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region,activity_flag
729,168851,Male,South Carolina,30,1/1/2017,0,single,63712,1903574,1,2,8,NaN,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,3,New customer,3,7.485714,Low spender,20.5,Regular customer,South,low activity
730,168851,Male,South Carolina,30,1/1/2017,0,single,63712,1193140,2,3,14,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,3,7.485714,Low spender,20.5,Regular customer,South,low activity
731,168851,Male,South Carolina,30,1/1/2017,0,single,63712,1870356,3,1,8,26.0,6184,2,0,Clementines,32,4,4.3,Low-range product,Regularly busy,Busiest days,Average orders,3,New customer,3,7.485714,Low spender,20.5,Regular customer,South,low activity
732,168851,Male,South Carolina,30,1/1/2017,0,single,63712,1903574,1,2,8,NaN,130,3,0,Vanilla Milk Chocolate Almond Ice Cream Bars M...,37,1,8.2,Mid-range product,Regularly busy,Regularly busy,Average orders,3,New customer,3,7.485714,Low spender,20.5,Regular customer,South,low activity
733,168851,Male,South Carolina,30,1/1/2017,0,single,63712,1193140,2,3,14,15.0,130,2,1,Vanilla Milk Chocolate Almond Ice Cream Bars M...,37,1,8.2,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,3,7.485714,Low spender,20.5,Regular customer,South,low activity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,80148,Female,New York,55,4/1/2020,1,married,57095,2859858,3,6,11,12.0,38750,11,0,"Protein Bar, Chocolate Pretzel",3,19,4.7,Low-range product,Regularly busy,Regularly busy,Most orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer,Northeast,low activity
32404855,80148,Female,New York,55,4/1/2020,1,married,57095,2859858,3,6,11,12.0,39972,5,0,Chocolate Fudge High Protein Bar,3,19,4.0,Low-range product,Regularly busy,Regularly busy,Most orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer,Northeast,low activity
32404856,80148,Female,New York,55,4/1/2020,1,married,57095,3209855,4,6,19,7.0,3198,2,0,Pumpkin Spice Latte Iced Coffee Pure Cold Brew...,26,7,4.7,Low-range product,Regularly busy,Regularly busy,Average orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer,Northeast,low activity
32404857,80148,Female,New York,55,4/1/2020,1,married,57095,2859858,3,6,11,12.0,39069,2,0,Chocolate Coconut Almond Protein Bar,3,19,1.8,Low-range product,Regularly busy,Regularly busy,Most orders,4,New customer,4,3.886667,Low spender,12.0,Regular customer,Northeast,low activity


In [44]:
# The subset numbers add up, 30964564 active customers
df_active_customers

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region,activity_flag
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,2524893,3,3,11,30.0,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Slowest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404813,135902,Female,Missouri,66,4/1/2020,2,married,53374,974951,1,6,17,NaN,31136,1,0,Original Flavor Sugar Free Gum 3 Pk,46,19,4.9,Low-range product,Regularly busy,Regularly busy,Average orders,8,New customer,8,6.960345,Low spender,8.0,Frequent customer,Midwest,active customer
32404814,135902,Female,Missouri,66,4/1/2020,2,married,53374,628984,4,5,14,13.0,31136,9,1,Original Flavor Sugar Free Gum 3 Pk,46,19,4.9,Low-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,6.960345,Low spender,8.0,Frequent customer,Midwest,active customer
32404815,135902,Female,Missouri,66,4/1/2020,2,married,53374,1157983,7,5,15,7.0,31136,2,1,Original Flavor Sugar Free Gum 3 Pk,46,19,4.9,Low-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,6.960345,Low spender,8.0,Frequent customer,Midwest,active customer
32404816,135902,Female,Missouri,66,4/1/2020,2,married,53374,2498530,8,6,21,8.0,32312,7,0,Glide Pro-Health Comfort Plus Mint Floss,20,11,8.4,Mid-range product,Regularly busy,Regularly busy,Average orders,8,New customer,8,6.960345,Low spender,8.0,Frequent customer,Midwest,active customer


In [42]:
# Saving the sample/subset with low activity customers as a pickle. 
df_low_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_low_activity_cutomers.pkl'))

In [43]:
# Saving the sample/subset with active customers as a pickle. 
df_active_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_active_cutomers.pkl'))

# 05. Creating a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. 

### From here on (as requested to exclude customers with less than 5 orders - low activity customers), I will use the new created data set with customers that have more than 5 orders - active customers.

## Creating age groups

In [50]:
# Here we can see that the minimum age is 18 and the max is 81 years old. 
df_active_customers['age'].describe()

count    3.096456e+07
mean     4.946803e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: age, dtype: float64

### Given the statistics for the age group in our dataset, I did created these age groups: 18 to 29 years old, 30 to 44 years old, 45 to 59 years old, 60 to 81 years old. 

In [52]:
df_active_customers.loc[(df_active_customers['age'] >= 18) & (df_active_customers['age'] <= 29), 'age_group'] = '18-29 years old'


In [53]:
df_active_customers.loc[(df_active_customers['age'] >= 30) & (df_active_customers['age'] <= 44), 'age_group'] = '30-44 years old'


In [54]:
df_active_customers.loc[(df_active_customers['age'] >= 45) & (df_active_customers['age'] <= 59), 'age_group'] = '45-59 years old'


In [55]:
df_active_customers.loc[(df_active_customers['age'] >= 60) & (df_active_customers['age'] <= 81), 'age_group'] = '60-81 years old'

In [58]:
df_active_customers['age_group'].value_counts(dropna = False)

age_group
60-81 years old    10574504
45-59 years old     7311091
30-44 years old     7261366
18-29 years old     5817603
Name: count, dtype: int64

## Creating income groups

In [59]:
df_active_customers['income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

### Given the statistics for the income in our dataset, I did create these age groups: 'Low Income' i called under 67292, 'Middle Income' is over 67292 until 128102, and High Income' over 128102.


In [61]:
df_active_customers.loc[df_active_customers['income'] <= 67292, 'income_group'] = 'Low Income'


In [62]:
df_active_customers.loc[(df_active_customers['income'] > 67292) & (df_active_customers['income'] <= 128102), 'income_group'] = 'Middle Income'


In [63]:
df_active_customers.loc[df_active_customers['income'] > 128102, 'income_group'] = 'High Income'

In [64]:
df_active_customers.head()

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region,activity_flag,age_group,income_group
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,2524893,3,3,11,30.0,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,Mid-range product,Regularly busy,Slowest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income


In [65]:
df_active_customers['income_group'].value_counts(dropna=False)

income_group
Middle Income    15482125
Low Income        7741348
High Income       7741091
Name: count, dtype: int64

In [75]:
df_end = df_active_customers

## Creating labels based on "department_id"

In [6]:
# Downloading "departments_wrangled" sheet/set
df_dept = pd.read_csv(os.path.join(path,'02 Data', 'Prepared Data', 'departments_wrangled.csv'))

In [7]:
df_dept

,Unnamed: 0,departments
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [8]:
# Renaming Unnamed: 0' to 'department_id'
df_dept.rename(columns = {'Unnamed: 0' : 'department_id'}, inplace = True)

In [9]:
df_dept['department_id'].describe()

count    21.000000
mean     11.000000
std       6.204837
min       1.000000
25%       6.000000
50%      11.000000
75%      16.000000
max      21.000000
Name: department_id, dtype: float64

In [10]:
df_dept

,department_id,departments
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [11]:
# Merging the final data set with all the changes made with the departments file
df_end = df_end.merge(df_dept, on = 'department_id')

In [12]:
df_end.head()

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region,activity_flag,age_group,income_group,departments
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages
3,33890,Female,New Mexico,36,1/1/2017,0,single,59285,1044841,3,6,11,30.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,12,Regular customer,12,7.057534,Low spender,26.0,Regular customer,West,active customer,30-44 years old,Low Income,beverages
4,33890,Female,New Mexico,36,1/1/2017,0,single,59285,474720,4,1,20,16.0,196,6,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Average orders,12,Regular customer,12,7.057534,Low spender,26.0,Regular customer,West,active customer,30-44 years old,Low Income,beverages


## Creating number of dependents group

### Creating a dependents flag/group:

In [13]:
df_end.loc[df_end['n_dependants'] == 0, 'dependants_group'] = 'No dependants'

In [14]:
df_end.loc[df_end['n_dependants'] > 0, 'dependants_group'] = 'Has dependants'

In [15]:
# The number of dependats in the df
df_end['n_dependants'].value_counts()

n_dependants
3    7772516
0    7739681
2    7733261
1    7719106
Name: count, dtype: int64

In [16]:
# The number of dependats in the df as a group. The numbers add up. 
df_end['dependants_group'].value_counts()

dependants_group
Has dependants    23224883
No dependants      7739681
Name: count, dtype: int64

In [17]:
df_end.head()

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region,activity_flag,age_group,income_group,departments,dependants_group
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages,Has dependants
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages,Has dependants
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages,Has dependants
3,33890,Female,New Mexico,36,1/1/2017,0,single,59285,1044841,3,6,11,30.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,12,Regular customer,12,7.057534,Low spender,26.0,Regular customer,West,active customer,30-44 years old,Low Income,beverages,No dependants
4,33890,Female,New Mexico,36,1/1/2017,0,single,59285,474720,4,1,20,16.0,196,6,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Average orders,12,Regular customer,12,7.057534,Low spender,26.0,Regular customer,West,active customer,30-44 years old,Low Income,beverages,No dependants


## Family Profile - Grouping

In [25]:
df_end['marital_status'].value_counts(dropna = False)

marital_status
married                             21743711
single                               5094410
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: count, dtype: int64

In [62]:
df_end['gender'].value_counts(dropna = False)

gender
Male      15586740
Female    15377824
Name: count, dtype: int64

### Married male & female with no children

In [52]:
df_end.loc[(df_end['marital_status'] == 'married') & (df_end['dependants_group'] == 'No dependants') & (df_end['gender'] == 'Female'), 'family_profile'] = 'married female, no children'

In [53]:
df_end.loc[(df_end['marital_status'] == 'married') & (df_end['dependants_group'] == 'No dependants') & (df_end['gender'] == 'Male'), 'family_profile'] = 'married male, no children'

### Married male & female with children

In [54]:
df_end.loc[(df_end['marital_status'] == 'married') & (df_end['dependants_group'] == 'Has dependants') & (df_end['gender'] == 'Female'), 'family_profile'] = 'married female with children'

In [55]:
df_end.loc[(df_end['marital_status'] == 'married') & (df_end['dependants_group'] == 'Has dependants') & (df_end['gender'] == 'Male'), 'family_profile'] = 'married male with children'

### Single male & female with no children 

In [56]:
df_end.loc[(df_end['marital_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_end['dependants_group'] == 'No dependants') & (df_end['gender'] == 'Female'), 'family_profile'] = 'single female, no children'

In [57]:
df_end.loc[(df_end['marital_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_end['dependants_group'] == 'No dependants') & (df_end['gender'] == 'Male'), 'family_profile'] = 'single male, no children'

### Single male & female with children 

In [58]:
df_end.loc[(df_end['marital_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_end['dependants_group'] == 'Has dependants') & (df_end['gender'] == 'Female'), 'family_profile'] = 'single female with children'

In [59]:
df_end.loc[(df_end['marital_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_end['dependants_group'] == 'Has dependants') & (df_end['gender'] == 'Male'), 'family_profile'] = 'single male with children'

In [60]:
# Based on the results we can see that the numbers make sense in general, but there is no result of married male or female without children.
df_end['family_profile'].value_counts(dropna = False)

family_profile
married male with children      10947746
married female with children    10795965
single male, no children         3881398
single female, no children       3858283
single male with children         757596
single female with children       723576
Name: count, dtype: int64

In [61]:
df_end.head()

,user_id,gender,state,age,date_joined,n_dependants,marital_status,income,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg,avg_price,spending_flag,customer_order_frequency,order_frequency_flag,region,activity_flag,age_group,income_group,departments,dependants_group,family_profile
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages,Has dependants,married female with children
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages,Has dependants,married female with children
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,8,New customer,8,7.988889,Low spender,19.0,Regular customer,Midwest,active customer,45-59 years old,High Income,beverages,Has dependants,married female with children
3,33890,Female,New Mexico,36,1/1/2017,0,single,59285,1044841,3,6,11,30.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,12,Regular customer,12,7.057534,Low spender,26.0,Regular customer,West,active customer,30-44 years old,Low Income,beverages,No dependants,"single female, no children"
4,33890,Female,New Mexico,36,1/1/2017,0,single,59285,474720,4,1,20,16.0,196,6,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Average orders,12,Regular customer,12,7.057534,Low spender,26.0,Regular customer,West,active customer,30-44 years old,Low Income,beverages,No dependants,"single female, no children"


In [64]:
df_end.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_end_final.pkl'))